In [2]:
from transformers import BartForConditionalGeneration, AutoTokenizer
bart = BartForConditionalGeneration.from_pretrained('hyunwoongko/kobart')
tokenizer = AutoTokenizer.from_pretrained('hyunwoongko/kobart')

In [3]:
texts = ['최근 들어 딥러닝', '안녕 친구야']
sample_ids = tokenizer(texts, padding=True, return_tensors='pt')['input_ids']
sample_ids

tensor([[19261, 14301,  1700, 10021, 10277,  9747],
        [11699,  9592, 15230, 11734,     3,     3]])

In [4]:
outs = bart.generate(
    sample_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=1,
    early_stopping=True,
)

In [31]:
import torch
import torch.nn.functional as F

def rerank(matrix, repeat_time, margin=0.):
    upper = matrix.unsqueeze(-1).repeat_interleave(repeat_time, dim=-1)
    lower = matrix.unsqueeze(1).repeat_interleave(repeat_time, dim=1)
    return F.relu((lower - upper + margin).triu(1))

In [32]:
batch_size = 8
num_cands = 16
# # embed_size = docs['input_ids'].size(-1)
# assert num_cands == cands['input_ids'].size(0) // batch_size

# doc_embeddings = self.encoder(**docs)[0][:, 0, :]
# gold_embeddings = self.encoder(**golds)[0][:, 0, :]
# sims_doc_gold = torch.cosine_similarity(doc_embeddings, gold_embeddings, dim=-1)

# cand_embeddings = self.encoder(**cands)[0][:, 0, :].view(batch_size, num_cands, -1)
# doc_embeddings = doc_embeddings.unsqueeze(1).repeat_interleave(num_cands, dim=1)
# sims_doc_cand = torch.cosine_similarity(doc_embeddings, cand_embeddings, dim=-1)

# scores_gold = sims_doc_cand - sims_doc_gold.unsqueeze(1).repeat_interleave(num_cands, dim=1)
# scores_gold = F.relu(scores_gold).sum(1)

ranks = (
    torch.arange(num_cands)
    .unsqueeze(0)
    .repeat_interleave(batch_size, dim=0)
)
rerank(ranks, num_cands)

tensor([[[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.]

In [33]:
import pandas as pd
pd.DataFrame({'ab': [[1,2,3],[1,2,3]]})

,ab
0,"[1, 2, 3]"
1,"[1, 2, 3]"


In [34]:
from datasets import load_metric
rouge = load_metric('rouge')

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [47]:
predictions = ['abc ccc', 'edf']
references = ['abc ccc', 'edf']
# ['최근 들어 딥러닝은 음성, 시각, 자연어, 로봇, 게임 등 인공지능 여러 분야의', '환경을 근본적으로 변화시키고 있다.']
# references = ['최근 들어 딥러닝은 음성, 시각, 자연어, 로봇, 게임 등 인공지능 여러 분야의', '환경을 근본적으로 변화시키고 있다.']
scores = rouge.compute(predictions=predictions, references=references[::-1])

In [48]:
scores['rouge1'].mid.fmeasure

0.0

In [1]:
valid_path = '/Volumes/GoogleDrive/My Drive/gassum/data/valid_w_cands.csv'

In [2]:
import pandas as pd

In [3]:
valid = pd.read_csv(valid_path)

In [5]:
valid = valid.iloc[:, 2:]

In [15]:
eval(valid['candidates'][1])

['바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하는 등 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.',
 '바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하는 등 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있는 것으로 알려졌다.',
 '바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하여 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.',
 '바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하는 등, 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.']

In [16]:
train_path = '/Volumes/GoogleDrive/My Drive/gassum/data/train_w_cands.csv'

In [44]:
train = pd.read_csv(train_path)

KeyboardInterrupt: 

In [18]:
len(train)

271093

In [20]:
train = train.iloc[:, 2:]

In [40]:
train.to_csv(train_path, index=False)

In [39]:
valid.loc[~valid['summary'].map(lambda x: isinstance(x, str))]

,id,title,text,summary,publish_date,size,char_count,category,media_type,media_sub_type,media_name,candidates


In [45]:
valid.loc[valid['candidates'].map(lambda x: len(eval(x)) != 4)]

,id,title,text,summary,publish_date,size,char_count,category,media_type,media_sub_type,media_name,candidates


In [46]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained('kykim/electra-kor-base')

In [47]:
config.embedding_size

768

In [49]:
import jsonlines
def load_eval_data_from_jsonl(path):
    table = {'id': [], 'article': [], 'media': []}
    with open(path) as f:
        with jsonlines.Reader(f) as reader:
            for obj in reader.iter():
                table['id'].append(obj['id'])
                table['article'].append(' '.join(obj['article_original']))
                table['media'].append(obj['media'])
    return pd.DataFrame(data=table)

In [51]:
test = load_eval_data_from_jsonl('/Volumes/GoogleDrive/My Drive/gassum/data/new_test.jsonl')

In [53]:
test['text'] = test['article']
del test['article']
test

,id,media,text
0,472779091,중앙일보,일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(65) 전 르노닛산 회장이...
1,472779092,중앙일보,본선에서 트럼프에 패할 수 있다는 우려 때문이다. 지지율 3위: 엘리자베스 워런 ◈...
2,472779093,중앙일보,1월 1일 새해 김정은 북한 국무위원장은 모습을 드러내지 않았다. 2012년 집권 ...
3,472779094,중앙일보,국내 이동통신 3사(KT·SK텔레콤·LG유플러스)는 산토끼(신규 고객)를 잡느라 집...
4,472779095,중앙일보,카를로스 곤(66)닛산자동차 전 회장의 ‘보석기간 중 탈출극’으로 인한 일본내 충격...
...,...,...,...
6591,472785682,환경부,"제1조(목적)"" ""이 고시는 「온실가스 배출권의 할당 및 거래에 관한 법률」(이하 ..."
6592,472785683,헤럴드경제,산업통상자원부와 한국가스안전공사는 겨울철 불법 가스용품 유통에 따른 가스사고 예방을...
6593,472785684,머니투데이,정부가 올해 안에 2050 탄소중립 시나리오와 2030년 국가 온실가스감축목표(ND...
6594,472785685,경향신문,"정부가 수소차 보급 활성화를 위해 수소 제조용 천연가스 요금을 한시적으로 낮춘다.""..."


In [54]:
test[['id', 'text', 'media']]

,id,text,media
0,472779091,일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(65) 전 르노닛산 회장이...,중앙일보
1,472779092,본선에서 트럼프에 패할 수 있다는 우려 때문이다. 지지율 3위: 엘리자베스 워런 ◈...,중앙일보
2,472779093,1월 1일 새해 김정은 북한 국무위원장은 모습을 드러내지 않았다. 2012년 집권 ...,중앙일보
3,472779094,국내 이동통신 3사(KT·SK텔레콤·LG유플러스)는 산토끼(신규 고객)를 잡느라 집...,중앙일보
4,472779095,카를로스 곤(66)닛산자동차 전 회장의 ‘보석기간 중 탈출극’으로 인한 일본내 충격...,중앙일보
...,...,...,...
6591,472785682,"제1조(목적)"" ""이 고시는 「온실가스 배출권의 할당 및 거래에 관한 법률」(이하 ...",환경부
6592,472785683,산업통상자원부와 한국가스안전공사는 겨울철 불법 가스용품 유통에 따른 가스사고 예방을...,헤럴드경제
6593,472785684,정부가 올해 안에 2050 탄소중립 시나리오와 2030년 국가 온실가스감축목표(ND...,머니투데이
6594,472785685,"정부가 수소차 보급 활성화를 위해 수소 제조용 천연가스 요금을 한시적으로 낮춘다.""...",경향신문


In [55]:
test = test[['id', 'text', 'media']]

In [56]:
test.to_csv('/Volumes/GoogleDrive/My Drive/gassum/data/new_test.csv')